In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [23]:
age_counts = {12: 5, 13: 5, 14: 5, 15: 5, 16: 5, 17: 5, 18: 5}
binge_percentages = {12: 1, 13: 3, 14: 6, 15: 14, 16: 36, 17: 48, 18: 86}
num_steps = 28
num_epochs = 1000

# Special event frequency (5-10% of epochs)
special_event_probability = 0.07

# Hardcoded drinker pools per age
fixed_drinker_counts = {12: 3, 13: 3, 14: 3, 15: 3, 16: 4, 17: 4, 18: 5}

# Initialize drinker status with some drinking frequently, others rarely
drinkers = {}
drinking_patterns = {}
ping_tracker = {age: np.zeros((age_counts[age], num_epochs), dtype=int) for age in age_counts}

for age, count in age_counts.items():
    num_drinkers = fixed_drinker_counts[age]
    
    drinkers[age] = np.zeros(count, dtype=bool)
    drinkers[age][:num_drinkers] = True
    np.random.shuffle(drinkers[age])
    
    # Assign drinking frequencies (frequent vs. rare drinkers)
    drinking_patterns[age] = np.zeros(count, dtype=float)
    for i in range(num_drinkers):
        drinking_patterns[age][i] = np.random.uniform(0.1, 0.9)  # Assign probability of drinking

In [24]:
# Allocate memory for ping data
players_pinged = {age: np.zeros(num_epochs, dtype=int) for age in age_counts}

def adaptive_ping(age, epoch, special_event):
    """Determine the number of players that should ping in this epoch."""
    possible_drinkers = np.where(drinkers[age])[0]
    
    if special_event:
        return possible_drinkers  # All drinkers ping during special events
    
    # Select drinkers probabilistically based on their assigned drinking patterns
    selected = [i for i in possible_drinkers if np.random.rand() < drinking_patterns[age][i]]
    
    return selected

for epoch in range(num_epochs):
    special_event = np.random.rand() < special_event_probability  # Determine if epoch is a special event
    for age in age_counts:
        selected_pingers = adaptive_ping(age, epoch, special_event)
        players_pinged[age][epoch] = len(selected_pingers)  # Store count
        for player in selected_pingers:
            ping_tracker[age][player, epoch] += 1  # Increment ping count for the player

In [25]:
# Normalize Results
normalized_ping_percentage = {}
for age in age_counts:
    total_pings = np.sum(players_pinged[age])
    expected_pings = (binge_percentages[age] / 100.0) * num_epochs * np.sum(drinkers[age])
    normalization_factor = expected_pings / total_pings if total_pings > 0 else 1
    normalized_ping_percentage[age] = (total_pings * normalization_factor) / (num_epochs * np.sum(drinkers[age])) * 100

# Print results with per-player ping counts and min/max pings per epoch
for age in age_counts:
    print(f"Age {age}: {normalized_ping_percentage[age]:.2f}% pings (target: {binge_percentages[age]}%)")
    for player in range(age_counts[age]):
        total_pings = np.sum(ping_tracker[age][player, :])
        min_pings = np.min(ping_tracker[age][player, :])
        max_pings = np.max(ping_tracker[age][player, :])
        print(f"  Player {player} pinged {total_pings} times over {num_epochs} epochs. Min ping: {min_pings}, Max ping: {max_pings}")


Age 12: 1.00% pings (target: 1%)
  Player 0 pinged 759 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 1 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 2 pinged 909 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 3 pinged 56 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 4 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
Age 13: 3.00% pings (target: 3%)
  Player 0 pinged 645 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 1 pinged 440 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 2 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 3 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 4 pinged 56 times over 1000 epochs. Min ping: 0, Max ping: 1
Age 14: 6.00% pings (target: 6%)
  Player 0 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 1 pinged 324 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 2 pinged 0 times over 1000 epochs. Min ping: 0, Max 

In [26]:
plt.figure(figsize=(20, 10))
n_ages = len(age_counts)
cmap = get_cmap('RdYlGn_r')
for i, (age, player_count) in enumerate(age_counts.items(), 1):
    ax = plt.subplot(1, n_ages, i)
    pinged_counts = players_pinged[age]
    norm = plt.Normalize(vmin=0, vmax=player_count)
    scatter = ax.scatter(pinged_counts, np.arange(num_epochs), 
                         c=pinged_counts, cmap=cmap, norm=norm, 
                         s=10, alpha=0.7)
    ax.set_title(f"{age} jaar")
    ax.set_xlabel("Aantal agents gepingt")
    ax.set_ylabel("Generaties")
    ax.set_xlim(-1, player_count + 1)
    ax.set_ylim(-50, num_epochs + 50)
    ax.set_yticks(np.arange(0, num_epochs + 1, 100))
    ax.set_xticks([0, 1, 2, 3, 4, 5])
    ax.grid(True)
plt.tight_layout()
os.makedirs(output_dir, exist_ok=True)
filename = "Aantal agent per eeftijd die willekeurig pingen over 1000 tests.png"
plt.savefig(f"{filename}", dpi=300, bbox_inches='tight')
plt.close()
print(f"opgeslagen")

/var/folders/9x/4jq4r89s0yl_53d6vmg31bcr0000gn/T/ipykernel_1627/2894256082.py:3: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = get_cmap('RdYlGn_r')


opgeslagen


In [27]:
import numpy as np

age_counts = {12: 5, 13: 5, 14: 5, 15: 5, 16: 5, 17: 5, 18: 5}
binge_percentages = {12: 0.8, 13: 2.5, 14: 5.5, 15: 13, 16: 37, 17: 50, 18: 88}
num_epochs = 1000
special_event_probability = 0.07
fixed_drinker_counts = {12: 3, 13: 3, 14: 3, 15: 3, 16: 4, 17: 4, 18: 5}

drinkers = {}
ping_tracker = {age: np.zeros((age_counts[age], num_epochs), dtype=int) for age in age_counts}
players_pinged = {age: np.zeros(num_epochs, dtype=int) for age in age_counts}

def gaussian_pings(mean_percentage, num_drinkers, num_epochs):
    mean_pings = mean_percentage / 100 * num_epochs * num_drinkers
    std_dev = max(1, mean_pings * 0.12)
    raw_pings = np.random.normal(mean_pings / num_epochs, std_dev / num_epochs, num_epochs).astype(int)
    raw_pings = np.clip(raw_pings, 0, num_drinkers)
    
    total_pings = raw_pings.sum()
    target_total = int(mean_pings)
    adjustment = target_total - total_pings
    
    while adjustment != 0:
        epoch_indices = np.random.choice(num_epochs, abs(adjustment), replace=True)
        for idx in epoch_indices:
            if adjustment > 0 and raw_pings[idx] < num_drinkers:
                raw_pings[idx] += 1
                adjustment -= 1
            elif adjustment < 0 and raw_pings[idx] > 0:
                raw_pings[idx] -= 1
                adjustment += 1
            if adjustment == 0:
                break
    
    return raw_pings

for age, count in age_counts.items():
    num_drinkers = fixed_drinker_counts[age]
    drinkers[age] = np.zeros(count, dtype=bool)
    drinkers[age][:num_drinkers] = True
    np.random.shuffle(drinkers[age])

for age in age_counts:
    ping_distribution = gaussian_pings(binge_percentages[age], fixed_drinker_counts[age], num_epochs)
    drinker_indices = np.where(drinkers[age])[0]
    for epoch, count in enumerate(ping_distribution):
        if count > 0:
            selected = np.random.choice(drinker_indices, count, replace=False)
            players_pinged[age][epoch] = count
            for player in selected:
                ping_tracker[age][player, epoch] += 1


# Print results with per-player ping counts and min/max pings per epoch
for age in age_counts:
    print(f"Age {age}: {normalized_ping_percentage[age]:.2f}% pings (target: {binge_percentages[age]}%)")
    for player in range(age_counts[age]):
        total_pings = np.sum(ping_tracker[age][player, :])
        min_pings = np.min(ping_tracker[age][player, :])
        max_pings = np.max(ping_tracker[age][player, :])
        print(f"  Player {player} pinged {total_pings} times over {num_epochs} epochs. Min ping: {min_pings}, Max ping: {max_pings}")


Age 12: 1.00% pings (target: 0.8%)
  Player 0 pinged 8 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 1 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 2 pinged 7 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 3 pinged 9 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 4 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
Age 13: 3.00% pings (target: 2.5%)
  Player 0 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 1 pinged 24 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 2 pinged 24 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 3 pinged 27 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 4 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
Age 14: 6.00% pings (target: 5.5%)
  Player 0 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 1 pinged 53 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 2 pinged 53 times over 1000 epochs. Min ping: 0, Max p

In [21]:
import numpy as np

age_counts = {12: 5, 13: 5, 14: 5, 15: 5, 16: 5, 17: 5, 18: 5}
binge_percentages = {12: 1, 13: 3, 14: 6, 15: 14, 16: 36, 17: 48, 18: 86}
num_epochs = 1000
special_event_probability = 0.07
fixed_drinker_counts = {12: 3, 13: 3, 14: 3, 15: 3, 16: 4, 17: 4, 18: 5}

drinkers = {}
ping_tracker = {age: np.zeros((age_counts[age], num_epochs), dtype=int) for age in age_counts}
players_pinged = {age: np.zeros(num_epochs, dtype=int) for age in age_counts}

def gaussian_pings(mean_percentage, num_drinkers, num_epochs):
    mean_pings = mean_percentage / 100 * num_epochs * num_drinkers
    std_dev = max(1, mean_pings * 0.1)
    raw_pings = np.random.normal(mean_pings / num_epochs, std_dev / num_epochs, num_epochs).astype(int)
    raw_pings = np.clip(raw_pings, 0, num_drinkers)
    
    total_pings = raw_pings.sum()
    target_total = int(mean_pings)
    adjustment = target_total - total_pings
    
    while adjustment != 0:
        epoch_indices = np.random.choice(num_epochs, abs(adjustment), replace=True)
        for idx in epoch_indices:
            if adjustment > 0 and raw_pings[idx] < num_drinkers:
                raw_pings[idx] += 1
                adjustment -= 1
            elif adjustment < 0 and raw_pings[idx] > 0:
                raw_pings[idx] -= 1
                adjustment += 1
            if adjustment == 0:
                break
    
    return raw_pings

for age, count in age_counts.items():
    num_drinkers = fixed_drinker_counts[age]
    drinkers[age] = np.zeros(count, dtype=bool)
    drinkers[age][:num_drinkers] = True
    np.random.shuffle(drinkers[age])

for age in age_counts:
    ping_distribution = gaussian_pings(binge_percentages[age], fixed_drinker_counts[age], num_epochs)
    drinker_indices = np.where(drinkers[age])[0]
    for epoch, count in enumerate(ping_distribution):
        if count > 0:
            selected = np.random.choice(drinker_indices, count, replace=False)
            players_pinged[age][epoch] = count
            for player in selected:
                ping_tracker[age][player, epoch] += 1


In [28]:
import numpy as np

age_counts = {12: 5, 13: 5, 14: 5, 15: 5, 16: 5, 17: 5, 18: 5}
binge_percentages = {12: 0.8, 13: 2.5, 14: 5.5, 15: 13, 16: 37, 17: 50, 18: 88}
num_epochs = 1000
special_event_probability = 0.07
fixed_drinker_counts = {12: 3, 13: 3, 14: 3, 15: 3, 16: 4, 17: 4, 18: 5}

drinkers = {}
ping_tracker = {age: np.zeros((age_counts[age], num_epochs), dtype=int) for age in age_counts}
players_pinged = {age: np.zeros(num_epochs, dtype=int) for age in age_counts}

def gaussian_pings(mean_percentage, num_drinkers, num_epochs):
    mean_pings = mean_percentage / 100 * num_epochs * num_drinkers
    std_dev = max(1, mean_pings * 0.12)
    raw_pings = np.random.normal(mean_pings / num_epochs, std_dev / num_epochs, num_epochs).astype(int)
    raw_pings = np.clip(raw_pings, 0, num_drinkers)
    
    total_pings = raw_pings.sum()
    target_total = int(mean_pings)
    adjustment = target_total - total_pings
    
    while adjustment != 0:
        epoch_indices = np.random.choice(num_epochs, abs(adjustment), replace=True)
        for idx in epoch_indices:
            if adjustment > 0 and raw_pings[idx] < num_drinkers:
                raw_pings[idx] += 1
                adjustment -= 1
            elif adjustment < 0 and raw_pings[idx] > 0:
                raw_pings[idx] -= 1
                adjustment += 1
            if adjustment == 0:
                break
    
    return raw_pings

for age, count in age_counts.items():
    num_drinkers = fixed_drinker_counts[age]
    drinkers[age] = np.zeros(count, dtype=bool)
    drinkers[age][:num_drinkers] = True
    np.random.shuffle(drinkers[age])

for age in age_counts:
    ping_distribution = gaussian_pings(binge_percentages[age], fixed_drinker_counts[age], num_epochs)
    drinker_indices = np.where(drinkers[age])[0]
    for epoch, count in enumerate(ping_distribution):
        if count > 0:
            selected = np.random.choice(drinker_indices, count, replace=False)
            players_pinged[age][epoch] = count
            for player in selected:
                ping_tracker[age][player, epoch] += 1


# Print results with per-player ping counts and min/max pings per epoch
for age in age_counts:
    print(f"Age {age}: {normalized_ping_percentage[age]:.2f}% pings (target: {binge_percentages[age]}%)")
    for player in range(age_counts[age]):
        total_pings = np.sum(ping_tracker[age][player, :])
        min_pings = np.min(ping_tracker[age][player, :])
        max_pings = np.max(ping_tracker[age][player, :])
        print(f"  Player {player} pinged {total_pings} times over {num_epochs} epochs. Min ping: {min_pings}, Max ping: {max_pings}")


Age 12: 1.00% pings (target: 0.8%)
  Player 0 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 1 pinged 9 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 2 pinged 5 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 3 pinged 10 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 4 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
Age 13: 3.00% pings (target: 2.5%)
  Player 0 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 1 pinged 16 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 2 pinged 27 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 3 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 4 pinged 32 times over 1000 epochs. Min ping: 0, Max ping: 1
Age 14: 6.00% pings (target: 5.5%)
  Player 0 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 1 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 2 pinged 50 times over 1000 epochs. Min ping: 0, Max p

In [29]:
plt.figure(figsize=(20, 10))
n_ages = len(age_counts)
cmap = get_cmap('RdYlGn_r')
for i, (age, player_count) in enumerate(age_counts.items(), 1):
    ax = plt.subplot(1, n_ages, i)
    pinged_counts = players_pinged[age]
    norm = plt.Normalize(vmin=0, vmax=player_count)
    scatter = ax.scatter(pinged_counts, np.arange(num_epochs), 
                         c=pinged_counts, cmap=cmap, norm=norm, 
                         s=10, alpha=0.7)
    ax.set_title(f"{age} jaar")
    ax.set_xlabel("Aantal agents gepingt")
    ax.set_ylabel("Generaties")
    ax.set_xlim(-1, player_count + 1)
    ax.set_ylim(-50, num_epochs + 50)
    ax.set_yticks(np.arange(0, num_epochs + 1, 100))
    ax.set_xticks([0, 1, 2, 3, 4, 5])
    ax.grid(True)
plt.tight_layout()
os.makedirs(output_dir, exist_ok=True)
filename = "Aantal agent per eeftijd die willekeurig pingen over 1000 tests.png"
plt.savefig(f"{filename}", dpi=300, bbox_inches='tight')
plt.close()
print(f"opgeslagen")

/var/folders/9x/4jq4r89s0yl_53d6vmg31bcr0000gn/T/ipykernel_1627/2894256082.py:3: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = get_cmap('RdYlGn_r')


opgeslagen


In [3]:
import numpy as np

age_counts = {12: 5, 13: 5, 14: 5, 15: 5, 16: 5, 17: 5, 18: 5}
binge_percentages = {12: 0.8, 13: 2.5, 14: 5.5, 15: 13, 16: 37, 17: 50, 18: 88}
num_epochs = 1000
special_event_probability = 0.07
fixed_drinker_counts = {12: 3, 13: 3, 14: 3, 15: 3, 16: 4, 17: 4, 18: 5}

drinkers = {}
drink_resistance = {}
ping_tracker = {age: np.zeros((age_counts[age], num_epochs), dtype=int) for age in age_counts}
players_pinged = {age: np.zeros(num_epochs, dtype=int) for age in age_counts}

def gaussian_pings(mean_percentage, num_drinkers, num_epochs):
    mean_pings = mean_percentage / 100 * num_epochs * num_drinkers
    std_dev = max(1, mean_pings * 0.12)
    raw_pings = np.random.normal(mean_pings / num_epochs, std_dev / num_epochs, num_epochs).astype(int)
    raw_pings = np.clip(raw_pings, 0, num_drinkers)
    
    total_pings = raw_pings.sum()
    target_total = int(mean_pings)
    adjustment = target_total - total_pings
    
    while adjustment != 0:
        epoch_indices = np.random.choice(num_epochs, abs(adjustment), replace=True)
        for idx in epoch_indices:
            if adjustment > 0 and raw_pings[idx] < num_drinkers:
                raw_pings[idx] += 1
                adjustment -= 1
            elif adjustment < 0 and raw_pings[idx] > 0:
                raw_pings[idx] -= 1
                adjustment += 1
            if adjustment == 0:
                break
    
    return raw_pings

for age, count in age_counts.items():
    num_drinkers = fixed_drinker_counts[age]
    drinkers[age] = np.zeros(count, dtype=bool)
    drinkers[age][:num_drinkers] = True
    np.random.shuffle(drinkers[age])
    
    drink_resistance[age] = np.zeros(count)
    for i in range(num_drinkers):
        drink_resistance[age][i] = np.random.uniform(0.1, 0.9)

for age in age_counts:
    ping_distribution = gaussian_pings(binge_percentages[age], fixed_drinker_counts[age], num_epochs)
    drinker_indices = np.where(drinkers[age])[0]
    for epoch, count in enumerate(ping_distribution):
        if count > 0:
            weights = np.array([1.0 - drink_resistance[age][i] for i in drinker_indices])
            weights /= weights.sum()
            selected = np.random.choice(drinker_indices, count, replace=False, p=weights)
            players_pinged[age][epoch] = count
            for player in selected:
                ping_tracker[age][player, epoch] += 1

# Print results with per-player ping counts, min/max pings per epoch, and percentage of agents that pinged
for age in age_counts:
    print(f"Age {age}: {binge_percentages[age]}% target pings")
    for epoch in range(num_epochs):
        # Calculate the percentage of agents that pinged in the current epoch
        agents_pinged = np.sum(ping_tracker[age][:, epoch] > 0)
        percentage_pinged = (agents_pinged / age_counts[age]) * 100
        # print(f"  Epoch {epoch + 1}: {percentage_pinged:.2f}% of agents pinged")
    for player in range(age_counts[age]):
        total_pings = np.sum(ping_tracker[age][player, :])
        min_pings = np.min(ping_tracker[age][player, :])
        max_pings = np.max(ping_tracker[age][player, :])
        print(f"  Player {player} pinged {total_pings} times over {num_epochs} epochs. Min ping: {min_pings}, Max ping: {max_pings}")

Age 12: 0.8% target pings
  Player 0 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 1 pinged 1 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 2 pinged 5 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 3 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 4 pinged 18 times over 1000 epochs. Min ping: 0, Max ping: 1
Age 13: 2.5% target pings
  Player 0 pinged 18 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 1 pinged 23 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 2 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 3 pinged 34 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 4 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
Age 14: 5.5% target pings
  Player 0 pinged 58 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 1 pinged 37 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 2 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 3 pinged 7

In [32]:
plt.figure(figsize=(20, 10))
n_ages = len(age_counts)
cmap = get_cmap('RdYlGn_r')
for i, (age, player_count) in enumerate(age_counts.items(), 1):
    ax = plt.subplot(1, n_ages, i)
    pinged_counts = players_pinged[age]
    norm = plt.Normalize(vmin=0, vmax=player_count)
    scatter = ax.scatter(pinged_counts, np.arange(num_epochs), 
                         c=pinged_counts, cmap=cmap, norm=norm, 
                         s=10, alpha=0.7)
    ax.set_title(f"{age} jaar")
    ax.set_xlabel("Aantal agents gepingt")
    ax.set_ylabel("Generaties")
    ax.set_xlim(-1, player_count + 1)
    ax.set_ylim(-50, num_epochs + 50)
    ax.set_yticks(np.arange(0, num_epochs + 1, 100))
    ax.set_xticks([0, 1, 2, 3, 4, 5])
    ax.grid(True)
plt.tight_layout()
os.makedirs(output_dir, exist_ok=True)
filename = "Aantal agent per eeftijd die willekeurig pingen over 1000 tests.png"
plt.savefig(f"{filename}", dpi=300, bbox_inches='tight')
plt.close()
print(f"opgeslagen")

/var/folders/9x/4jq4r89s0yl_53d6vmg31bcr0000gn/T/ipykernel_1627/2894256082.py:3: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = get_cmap('RdYlGn_r')


opgeslagen
